Library imports

In [ ]:
import os
import cv2
import operator
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose, Dropout
from keras.models import Model
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras import optimizers
from google.colab import drive

Loading data

In [ ]:
!gdown https://drive.google.com/uc?id=1hNxU1Cht9SHiXLrqTDughcfxyYYeyB8R

In [ ]:
!unzip /content/Cat_Gan_2.zip 

Image preprocessing

In [ ]:
size1 = 96
size2 = 128

In [ ]:
os.mkdir("Training") #making the new Training and testing folders
dirs = ["Domino", "Stormy"] #Making the names for the subdirec
pardirs = ["/content/Training"]
for pardir in pardirs:
  for dir in dirs: 
    path = os.path.join(pardir, dir)
    os.mkdir(path)
datadir1 = "/content/Cat_Gan_2/Train/Domino"
datadir2 = "/content/Cat_Gan_2/Train/Stormy"
datadir3 = "/content/Cat_Gan_2/Test/Domino"
datadir4 = "/content/Cat_Gan_2/Test/Stormy"
filelist1 = sorted(os.listdir(datadir1), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist2 = sorted(os.listdir(datadir2), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist3 = sorted(os.listdir(datadir3), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist4 = sorted(os.listdir(datadir4), key = lambda fname: int(fname.split("_")[0][-4:]))
datadirs = [filelist1, filelist2, filelist3, filelist4]
inc = 0
idom = 0
istorm = 0
for filelist in datadirs:
  for fil in filelist:
    if inc == 0:
      path = "/content/Cat_Gan_2/Train/Domino/" + fil
      idom += 1
      img = cv2.imread(path)
      imgResized = cv2.resize(img, (size1, size2))
      cv2.imwrite('/content/Training/Domino/DominoTR%03i.jpg' %idom, imgResized)
    elif inc == 1:
      path = "/content/Cat_Gan_2/Train/Stormy/" + fil
      istorm += 1
      img = cv2.imread(path)
      imgResized = cv2.resize(img, (size1, size2))
      cv2.imwrite('/content/Training/Stormy/StormyTR%03i.jpg' %istorm, imgResized)
    elif inc == 2:
      path = "/content/Cat_Gan_2/Test/Domino/" + fil
      img = cv2.imread(path)
      idom += 1
      imgResized = cv2.resize(img, (size1, size2))
      cv2.imwrite('/content/Training/Domino/DominoTR%03i.jpg' %idom, imgResized)
    else:
      path = "/content/Cat_Gan_2/Test/Stormy/" + fil
      img = cv2.imread(path)
      istorm += 1
      imgResized = cv2.resize(img, (size1, size2))
      cv2.imwrite('/content/Training/Stormy/StormyTR%03i.jpg' %istorm, imgResized)
  inc += 1

In [ ]:
datadirDom = '/content/Training/Domino'
datadirStorm = '/content/Training/Stormy'
datadirDom = os.listdir(datadirDom)
datadirStorm = os.listdir(datadirStorm)
labels = []
cat_images = []
for fil in datadirDom:
  path = '/content/Training/Domino/' + fil
  image = Image.open(path)
  horz_image = np.array(image.transpose(method = Image.FLIP_LEFT_RIGHT))
  vert_image = np.array(image.transpose(method = Image.FLIP_TOP_BOTTOM))
  rot_image = np.array(image.rotate(180))
  image = np.array(image)
  cat_images.append(image)
  plt.imshow(image)
  cat_images.append(horz_image)
  cat_images.append(vert_image)
  cat_images.append(rot_image)
for fil in datadirStorm:
  path = '/content/Training/Stormy/' + fil
  image = Image.open(path)
  horz_image = np.array(image.transpose(method = Image.FLIP_LEFT_RIGHT))
  vert_image = np.array(image.transpose(method = Image.FLIP_TOP_BOTTOM))
  rot_image = np.array(image.rotate(180))
  image = np.array(image)
  cat_images.append(image)
  cat_images.append(horz_image)
  cat_images.append(vert_image)
  cat_images.append(rot_image)
cat_images = np.array(cat_images) / 255
print (cat_images.shape)

Defining Cat Autoencoder Class

In [ ]:
class CatAutoencoder:
  def __init__(self, latent_dim, img_size):
    self.latent_dim = latent_dim
    self.img_size = img_size
    self.encoder = self.build_encoder()
    self.decoder = self.build_decoder()

    inputs = Input(shape=(self.img_size[0], self.img_size[1], 3))

    output = self.decoder(self.encoder(inputs))
    self.autoencoder = Model(inputs, output, name='Cat_Autoencoder')

    self.autoencoder.compile(loss='mae', optimizer='adam')
  
  def build_encoder(self):

    # Variables

    layer_filters = [128, 96]
    kernel_size = 6

    # Layers

    inputs = Input(shape=(self.img_size[0], self.img_size[1], 3))  # RGB Image
    result = inputs
    for filters in layer_filters:
      result = Conv2D(filters, kernel_size, strides=2, padding='same', activation='relu')(result)
    
    result = Flatten()(result)
    result = Dropout(0.4)(result)
    result = Dense(100)(result)
    result = Dropout(0.1)(result)
    latent = Dense(self.latent_dim, activation='sigmoid', name='latent_vector')(result)

    encoder = Model(inputs, latent, name='Cat_Encoder')
    print(encoder.summary())
    return encoder

  def build_decoder(self):

    # Variables

    layer_filters = [96, 128]
    kernel_size = 6

    # Layers

    inputs = Input(shape=self.latent_dim)
    result = Dense(int((self.img_size[0] * self.img_size[1] * 128) / 2**(len(layer_filters)*2)))(inputs)
    result = Reshape((int(self.img_size[0] / 2**len(layer_filters)), int(self.img_size[1] / 2**len(layer_filters)), 128))(result)
    for filters in layer_filters:
      result = Conv2DTranspose(filters, kernel_size, activation='relu', padding='same', strides=2)(result)
    outputs = Conv2DTranspose(3, kernel_size, activation='sigmoid', padding='same')(result)

    decoder = Model(inputs, outputs, name='Cat_Decoder')
    print(decoder.summary())
    return decoder
  
  def train(self, images, episodes=500, batch_size=50, gen_interval=100, save_model=True, plot_graphs=True, verbose=True):
    tr_images, ts_images = train_test_split(images)
    epochs = []
    losses = []
    val_losses = []
    
    for i in range(int(episodes/gen_interval)):
      history = self.autoencoder.fit(tr_images, tr_images, validation_data=(ts_images, ts_images), epochs=gen_interval, batch_size=batch_size, verbose=verbose)
      for j in range(gen_interval):
        epochs.append(gen_interval*i + j + 1)
      for loss in history.history['loss']:
        losses.append(loss)
      for val_loss in history.history['val_loss']:
        val_losses.append(val_loss)
      decoded = self.autoencoder.predict(ts_images)
      plt.imshow(ts_images[0])
      plt.show()
      plt.imshow(decoded[0])
      plt.show()
      imgs = np.concatenate([ts_images[:8], decoded[:8]])
      imgs = imgs.reshape((4, 4, size2, size1, 3))
      imgs = np.vstack([np.hstack(i) for i in imgs])
      plt.figure()
      plt.axis('off')
      plt.title('%d: [Input: 1st 2 rows, Decoded: last 2 rows]' % (i+1))
      plt.imshow(imgs, interpolation='none', cmap='gray')
      plt.show()
    
    if plot_graphs:
      plt.scatter(epochs, losses, label='training_loss')
      plt.scatter(epochs, val_losses, label='testing_loss')
      plt.legend(loc='upper right')
      plt.xlabel('Epochs')
      plt.ylabel('Loss')
      plt.savefig('autoencoder_loss_graph.png')
      plt.show()
    
    if save_model:
      self.encoder.save('Cat_Encoder.h5')
      self.decoder.save('Cat_Decoder.h5')
      

Instantiating and Training Networks

In [ ]:
autoencoder = CatAutoencoder(100, (size2, size1))
autoencoder.train(cat_images)